# Quickstart

Heat a workshop with a gas boiler — the simplest possible system.

**Concepts**: Bus (balance node) · Flow (energy transfer) · Effect (cost tracking) · Port (source/sink) · Converter (efficiency coupling)

In [ ]:
from datetime import datetime

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from fluxopt import Bus, Converter, Effect, Flow, Port, optimize

pio.renderers.default = 'notebook_connected'

## System

```
Gas Grid ──► [gas] ──► Boiler η=90% ──► [heat] ──► Workshop
 0.08 €/kWh                                        30–50 kW
```

In [ ]:
timesteps = [datetime(2024, 1, 15, h) for h in range(8, 12)]

result = optimize(
    timesteps=timesteps,
    buses=[Bus('gas'), Bus('heat')],
    effects=[Effect('cost', unit='EUR', is_objective=True)],
    ports=[
        Port(
            'gas_grid',
            imports=[
                Flow(bus='gas', size=1000, effects_per_flow_hour={'cost': 0.08}),
            ],
        ),
        Port(
            'workshop',
            exports=[
                Flow(bus='heat', size=50, fixed_relative_profile=[0.6, 1.0, 0.9, 0.5]),
            ],
        ),
    ],
    converters=[
        Converter.boiler(
            'boiler',
            thermal_efficiency=0.9,
            fuel_flow=Flow(bus='gas', size=200),
            thermal_flow=Flow(bus='heat', size=100),
        ),
    ],
)

## Results

In [ ]:
demand = result.flow_rate('workshop(heat)')
total_heat = float(demand.sum())
print(f'Total cost: {result.objective:.2f} EUR  |  Avg: {result.objective / total_heat * 100:.1f} ct/kWh')

In [ ]:
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.15,
    subplot_titles=('Flow Rates', 'Cost per Timestep'),
)

times = result.flow_rates.coords['time'].values
for fid in result.flow_rates.coords['flow'].values:
    fig.add_trace(
        go.Scatter(x=times, y=result.flow_rate(str(fid)).values, name=str(fid), line_shape='hv'),
        row=1,
        col=1,
    )

cost = result.effects_temporal.sel(effect='cost')
fig.add_trace(go.Bar(x=times, y=cost.values, name='cost', showlegend=False), row=2, col=1)

fig.update_layout(height=450, margin={'l': 50, 'r': 20, 't': 30, 'b': 20}, template='plotly_white')
fig.update_yaxes(title_text='kW', row=1, col=1)
fig.update_yaxes(title_text='EUR', row=2, col=1)
fig

Flow ids are qualified as `component(bus)` — e.g. `boiler(gas)`, `workshop(heat)`.

**Next**: [Heat System with Storage](02-heat-system.ipynb) — time-varying prices and load shifting